# 3. Cassandra to Arrow

We use some code from the Cassandra server to read the SSTable, but instead of de/serializing to/from CQL, we use an [Arrow IPC stream](http://arrow.apache.org/), which is stored in a columnar format and better suited for analytics.

Data transformations:

1. SSTable on disk
2. Deserialized into Java Object in C* server
3. Client makes request to server (not to C* DB)
4. Data serialized via Arrow IPC stream
5. Sent across network
6. Arrow IPC stream received by client
7. Transformed into Arrow Table / cuDF

**Pros:**
- doesn't make request to the main Cassandra DB, which lessens the load and allows for other operations to run
- less de/serialization involved using the Arrow IPC stream

**Cons:**
- don't want to have to start Cassandra or use the JVM
- complex architecture

In [1]:
import pyarrow as pa
import pandas as pd
import socket

HOST = '127.0.0.1'
PORT = 9143

In [2]:
def read_bytes(sock, n):
    data = b''
    while len(data) < n:
        more = sock.recv(n - len(data))
        if not more:
            raise EOFError("Socket connection ended before reading specified number of bytes")
        data += more
    return data

def read_u8(sock):
    data = read_bytes(sock, 8)
    return int.from_bytes(data, byteorder='big')

# read data from socket
def fetch_data():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((HOST, PORT))
        sock.sendall(b'hello world\n')
        num_tables = read_u8(sock)
        table_buffers = []
        for i in range(num_tables):
            print('receiving table', i)
            table_size = read_u8(sock)
            buf = read_bytes(sock, table_size)
            table_buffers.append(buf)
    return table_buffers

In [3]:
buffers = fetch_data()
tables = [pa.ipc.open_stream(buf).read_all() for buf in buffers]
len(tables)

receiving table 0


1

In [4]:
table = tables[0].to_pandas()
table

,partition_key,_ts_row_liveness,_del_time_row_liveness,_ttl_row_liveness,_local_del_time_partition,_marked_for_del_at_partition,_local_del_time_row,_marked_for_delete_at_row,clustering_key_0,data,...,_del_time_data,_ttl_data,sensor_value,_ts_sensor_value,_del_time_sensor_value,_ttl_sensor_value,station_id,_ts_station_id,_del_time_station_id,_ttl_station_id
0,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.002,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.002,"ue sapien et, fermentum neque. Pellentesque mo...",...,NaT,NaT,95.759791,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
1,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.009,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.009,ctus mauris nec urna. Duis sit amet enim trist...,...,NaT,NaT,106.497951,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
2,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.002,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.002,ere purus vitae vehicula porttitor. Cum sociis...,...,NaT,NaT,100.090271,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
3,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.001,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.001,"luctus est, et vulputate odio felis interdum m...",...,NaT,NaT,94.181632,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
4,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.000,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.000,imperdiet sed est. Aliquam a quam condimentum...,...,NaT,NaT,99.909094,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.003,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.003,ulum vehicula euismod purus ac tempus. Nam mol...,...,NaT,NaT,105.890743,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
996,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.006,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.006,rat nibh. Vivamus eu quam id mi gravida porta ...,...,NaT,NaT,103.857088,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
997,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.000,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.000,pien.\nDonec sit amet est eu libero malesuada ...,...,NaT,NaT,95.420393,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT
998,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01 00:00:00.001,NaT,NaT,NaT,NaT,NaT,NaT,1970-01-01 00:00:00.001,"sem, gravida vitae commodo a, vestibulum plac...",...,NaT,NaT,103.726586,NaT,NaT,NaT,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda',NaT,NaT,NaT


In [7]:
table['sensor_value'][table['sensor_value'] != float('-inf')].min()

-inf

In [5]:
table['sensor_value'].mean()

-inf

In [ ]:
tables[1].to_pandas()

In [ ]:
tables[2].to_pandas()